In [ ]:
%matplotlib inline
# Bibliothèques Python
import os, cv2,itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob
from PIL import Image

# Bibliothèques PyTorch
import torch
from torch import optim,nn
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset
from torchvision import models,transforms

# Bibliothèques scikit-learn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Assurer la reproductibilité des résultats
np.random.seed(10)
torch.manual_seed(10)
torch.cuda.manual_seed(10)

# Vérification du contenu du répertoire
print(os.listdir('/kaggle/input/'))

In [ ]:
# Vérification du contenu du répertoire d'images
print(os.listdir('../input/skin-cancer-mnist-ham10000'))

In [ ]:
# Définir le répertoire des données
data_dir = '../input/skin-cancer-mnist-ham10000'

# Récupérer les chemins des images (fichiers .jpg)
all_image_path = glob(os.path.join(data_dir, '*', '*.jpg'))

# Créer un dictionnaire pour associer l'ID de l'image au chemin
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in all_image_path}

# Dictionnaire des types de lésions cutanées
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'dermatofibroma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

In [ ]:
import cv2
import numpy as np
from tqdm import tqdm

def compute_img_mean_std(image_paths):
    """
    Computing the mean and std of three channels for the whole dataset,
    first we should normalize the image from 0-255 to 0-1.
    
    Note: This is adjusted for InceptionV3 which requires 299x299 input size.
    """
    
    # Set the target size to 299x299 for InceptionV3
    img_h, img_w = 299, 299
    imgs = []
    means, stdevs = [], []

    # Loop through each image in the dataset
    for i in tqdm(range(len(image_paths))):
        # Read the image
        img = cv2.imread(image_paths[i])
        
        # Resize the image to 299x299
        img = cv2.resize(img, (img_w, img_h))
        
        # Append the resized image to the list
        imgs.append(img)

    # Stack images into a numpy array (shape: [height, width, channels, num_images])
    imgs = np.stack(imgs, axis=3)  # [H, W, C, N]
    print(imgs.shape)  # [299, 299, 3, N]

    # Convert images to float32 and normalize to [0, 1]
    imgs = imgs.astype(np.float32) / 255.0

    # Compute the mean and std for each channel (RGB)
    for i in range(3):  # Loop through the 3 channels (RGB)
        # Flatten the pixel values of the i-th channel
        pixels = imgs[:, :, i, :].ravel()  # Flatten to a 1D array
        means.append(np.mean(pixels))  # Calculate mean
        stdevs.append(np.std(pixels))  # Calculate standard deviation

    # Reverse the lists to match RGB order (OpenCV loads images in BGR)
    means.reverse()  
    stdevs.reverse()

    # Print the calculated mean and standard deviation values
    print("normMean = {}".format(means))
    print("normStd = {}".format(stdevs))

    # Return the mean and standard deviation
    return means, stdevs


In [ ]:
# Valeurs moyennes et écart-types normalisés pour les canaux R, G, B
norm_mean = [0.7630392, 0.5456477, 0.57004845]
norm_std = [0.1409286, 0.15261266, 0.16997074]

In [ ]:
# Chargement des métadonnées
df_original = pd.read_csv(os.path.join(data_dir, 'HAM10000_metadata.csv'))

# Ajout du chemin d'accès à chaque image
df_original['path'] = df_original['image_id'].map(imageid_path_dict.get)

# Ajout du type de lésion cutanée à partir du dictionnaire
df_original['cell_type'] = df_original['dx'].map(lesion_type_dict.get)

# Conversion du type de lésion en indices numériques
df_original['cell_type_idx'] = pd.Categorical(df_original['cell_type']).codes

df_original.head()

In [ ]:
# Regroupement par 'lesion_id' et comptage des occurrences
df_undup = df_original.groupby('lesion_id').count()

# Filtrer les 'lesion_id' qui ont seulement une image associée
df_undup = df_undup[df_undup['image_id'] == 1]

# Réinitialisation de l'index pour simplifier l'accès
df_undup.reset_index(inplace=True)

df_undup.head()

In [ ]:
# Fonction pour identifier les doublons dans les lésions
def get_duplicates(x):
    unique_list = list(df_undup['lesion_id'])
    if x in unique_list:
        return 'unduplicated'
    else:
        return 'duplicated'

# Création d'une nouvelle colonne qui est une copie de la colonne 'lesion_id'
df_original['duplicates'] = df_original['lesion_id']

# Application de la fonction pour marquer les doublons
df_original['duplicates'] = df_original['duplicates'].apply(get_duplicates)

df_original.head()

In [ ]:
# Comptage des valeurs dans la colonne 'duplicates' pour savoir combien de doublons et non-doublons il y a
df_original['duplicates'].value_counts().reset_index(name='count')

In [ ]:
# Filtrer les lésions qui sont marquées comme 'unduplicated'
df_undup = df_original[df_original['duplicates'] == 'unduplicated']
df_undup.shape

In [ ]:
y = df_undup['cell_type_idx']

# Création d'un ensemble de validation en utilisant train_test_split
_, df_val = train_test_split(df_undup, test_size=0.2, random_state=101, stratify=y)

# Affichage de la forme de l'ensemble de validation pour vérifier sa taille
df_val.shape

In [ ]:
# Vérification de la répartition des classes dans l'ensemble de validation
df_val['cell_type_idx'].value_counts().reset_index(name='count')

In [ ]:
def get_val_rows(x):
    # create a list of all the lesion_id's in the val set
    val_list = list(df_val['image_id'])
    if str(x) in val_list:
        return 'val'
    else:
        return 'train'

# Identify train and val rows
# Create a new colum that is a copy of the image_id column
df_original['train_or_val'] = df_original['image_id']
# Apply the function to this new column
df_original['train_or_val'] = df_original['train_or_val'].apply(get_val_rows)
# Filter out training rows
df_train = df_original[df_original['train_or_val'] == 'train']
print(len(df_train))
print(len(df_val))

In [ ]:
df_train['cell_type_idx'].value_counts().reset_index(name='count')

In [ ]:
df_val['cell_type'].value_counts().reset_index(name='count')

In [ ]:
import torch

# Vérifiez si CUDA (GPU) est disponible
cuda_available = torch.cuda.is_available()
print(f"CUDA is available: {cuda_available}")

# Si le GPU est disponible, utilisez-le, sinon utilisez le CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Affichez le nom du GPU, si disponible
if cuda_available:
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU found.")

# Exemple de tensor déplacé vers le GPU
tensor = torch.randn(3, 3).to(device)
print(f"Tensor on device: {tensor.device}")


In [ ]:
import pandas as pd

# Initialiser les taux d'augmentation souhaités
data_aug_rate = [15, 10, 5, 50, 0, 40, 5]

# Calculer le nombre maximum d'exemples parmi toutes les classes (pour le but d'équilibrage)
max_size = df_train['cell_type'].value_counts().max()

# Créer un DataFrame pour enregistrer les nouvelles données augmentées
augmented_data = []

# Boucle sur chaque classe et augmenter de manière équilibrée
for i in range(7):
    # Récupérer le sous-ensemble de données pour chaque classe
    class_data = df_train[df_train['cell_type_idx'] == i]
    
    # Calculer le nombre d'exemples à augmenter pour atteindre la taille cible
    target_size = max_size  # ou une taille cible définie manuellement
    current_size = len(class_data)
    augmentation_factor = (target_size // current_size)  # combien de fois multiplier les données de la classe
    
    # Si augmentation nécessaire
    if augmentation_factor > 1:
        augmented_class_data = pd.concat([class_data] * augmentation_factor, ignore_index=True)
        augmented_data.append(augmented_class_data)

# Concaténer toutes les classes augmentées avec les données d'origine
df_train_augmented = pd.concat([df_train] + augmented_data, ignore_index=True)

# Vérifier les nouvelles valeurs de 'cell_type' après augmentation
print(df_train_augmented['cell_type'].value_counts())

In [ ]:
# Split the test set again in a validation set and a true test set:
df_val, df_test = train_test_split(df_val, test_size=0.5)
df_train = df_train.reset_index()
df_val = df_val.reset_index()
df_test = df_test.reset_index()

print(len(df_test))
df_test['cell_type'].value_counts()

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

# Fonction pour initialiser le modèle
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    model_ft = None
    input_size = 0

    # Vérifier si le modèle demandé est 'inception'
    if model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        # Charger le modèle InceptionV3 avec des poids pré-entraînés (ImageNet par défaut)
        model_ft = models.inception_v3(pretrained=use_pretrained)

        # Fonction pour geler ou ajuster les paramètres du modèle selon l'option 'feature_extract'
        set_parameter_requires_grad(model_ft, feature_extract)

        # Traiter le réseau auxiliaire (AuxLogits)
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)

        # Traiter le réseau principal (fc)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)

        # Définir la taille d'entrée attendue par le modèle
        input_size = 299  # Taille d'entrée pour InceptionV3

    else:
        # Si le modèle spécifié n'est pas supporté, afficher un message d'erreur et arrêter l'exécution
        print("Nom du modèle invalide, sortie...")
        exit()

    # Retourner le modèle initialisé et la taille d'entrée
    return model_ft, input_size



In [ ]:
import torch

# Paramètres du modèle
model_name = "inception"  # Choix du modèle (ici InceptionV3)
num_classes = 7  # Nombre de classes pour la classification
feature_extract = False  # Indique si on veut uniquement extraire des caractéristiques (True) ou entraîner entièrement (False)

# Initialisation du modèle avec les paramètres définis ci-dessus
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Définir l'appareil (GPU ou CPU) sur lequel le modèle sera exécuté
device = torch.device('cuda:0')  # Utilisation du GPU (cuda:0 pour le premier GPU disponible)
# Si vous souhaitez utiliser le CPU au lieu du GPU, vous pouvez décommenter la ligne suivante :
# device = torch.device('cpu')  # Utilisation du CPU

# Déplacer le modèle sur l'appareil sélectionné (GPU ou CPU)
model = model_ft.to(device)

# Vérification que le modèle est bien sur l'appareil choisi
print(f"Le modèle est placé sur : {device}")


In [ ]:
# Définir les transformations pour les images d'entraînement
train_transform = transforms.Compose([transforms.Resize((input_size,input_size)),transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),
                                      transforms.RandomRotation(20),
                                      transforms.ColorJitter(brightness=0.1, contrast=0.1, hue=0.1),
                                      transforms.ToTensor(), 
                                      transforms.Normalize(norm_mean, norm_std)])

# Définir les transformations pour les images de validation
val_transform = transforms.Compose([transforms.Resize((input_size,input_size)), 
                                    transforms.ToTensor(),
                                    transforms.Normalize(norm_mean, norm_std)])

# Définir les transformations pour les images de test
test_transform = transforms.Compose([transforms.Resize((input_size,input_size)), 
                                     transforms.ToTensor(),
                                    transforms.Normalize(norm_mean, norm_std)])

In [ ]:
class HAM10000(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        # Load data and get label
        X = Image.open(self.df['path'][index])
        y = torch.tensor(int(self.df['cell_type_idx'][index]))

        if self.transform:
            X = self.transform(X)

        return X, y

In [ ]:
# Définir l'ensemble d'entraînement en utilisant le DataFrame train_df et les transformations définies (train_transform)
training_set = HAM10000(df_train, transform=train_transform)
train_loader = DataLoader(training_set, batch_size=32, shuffle=True, num_workers=4)

# Définir de la même manière l'ensemble de validation :
validation_set = HAM10000(df_val, transform=train_transform)
val_loader = DataLoader(validation_set, batch_size=32, shuffle=False, num_workers=4)

# Définir de la même manière l'ensemble de test :
test_set = HAM10000(df_test, transform=train_transform)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False, num_workers=4)


In [ ]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn, optim
from typing import Optional, List
from torch.nn.modules.loss import _Loss

# Constants for mode
BINARY_MODE = "binary"
MULTICLASS_MODE = "multiclass"
MULTILABEL_MODE = "multilabel"

# Dice Loss class
class DiceLoss(_Loss):
    def __init__(
        self,
        mode: str,
        classes: Optional[List[int]] = None,
        log_loss: bool = False,
        from_logits: bool = True,
        smooth: float = 0.0,
        ignore_index: Optional[int] = None,
        eps: float = 1e-7,
    ):
        assert mode in {BINARY_MODE, MULTILABEL_MODE, MULTICLASS_MODE}
        super(DiceLoss, self).__init__()
        self.mode = mode
        if classes is not None:
            assert mode != BINARY_MODE, "Masking classes is not supported with mode=binary"
            self.classes = torch.tensor(classes, dtype=torch.long)
        else:
            self.classes = None
        self.from_logits = from_logits
        self.smooth = smooth
        self.eps = eps
        self.log_loss = log_loss
        self.ignore_index = ignore_index

    def forward(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        assert y_true.size(0) == y_pred.size(0)

        if self.from_logits:
            if self.mode == MULTICLASS_MODE:
                y_pred = y_pred.log_softmax(dim=1).exp()
            else:
                y_pred = F.logsigmoid(y_pred).exp()

        bs = y_true.size(0)
        num_classes = y_pred.size(1)
        dims = (0, 2)

        if self.mode == BINARY_MODE:
            y_true = y_true.view(bs, 1, -1)
            y_pred = y_pred.view(bs, 1, -1)

            if self.ignore_index is not None:
                mask = y_true != self.ignore_index
                y_pred = y_pred * mask
                y_true = y_true * mask

        if self.mode == MULTICLASS_MODE:
            y_true = y_true.view(bs, -1)
            y_pred = y_pred.view(bs, num_classes, -1)

            if self.ignore_index is not None:
                mask = y_true != self.ignore_index
                y_pred = y_pred * mask.unsqueeze(1)
                y_true = F.one_hot((y_true * mask).to(torch.long), num_classes)
                y_true = y_true.permute(0, 2, 1) * mask.unsqueeze(1)
            else:
                y_true = F.one_hot(y_true, num_classes)
                y_true = y_true.permute(0, 2, 1)

        scores = self.compute_score(
            y_pred, y_true.type_as(y_pred), smooth=self.smooth, eps=self.eps, dims=dims
        )

        if self.log_loss:
            loss = -torch.log(scores.clamp_min(self.eps))
        else:
            loss = 1.0 - scores

        mask = y_true.sum(dims) > 0
        loss *= mask.to(loss.dtype)

        if self.classes is not None:
            loss = loss[self.classes]

        return loss.mean()

    def compute_score(
        self, output, target, smooth=0.0, eps=1e-7, dims=None
    ) -> torch.Tensor:
        intersection = torch.sum(output * target, dims)
        cardinality = torch.sum(output + target, dims)
        return (2. * intersection + smooth) / (cardinality + smooth + eps)


# Créez votre optimiseur et perte
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Assurez-vous que votre modèle Inception est sur le bon appareil

# Fonction de perte
criterion = DiceLoss(mode=MULTICLASS_MODE, smooth=1e-6).to(device)

# Optimiseur Adam
optimizer = optim.Adam(model.parameters(), lr=1e-4)

from torch.optim.lr_scheduler import StepLR

scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # Réduit le LR toutes les 5 époques


In [ ]:
total_loss_train, total_acc_train = [],[]
def train(train_loader, model, criterion, optimizer, epoch):
    model.train()
    train_loss = AverageMeter()
    train_acc = AverageMeter()
    curr_iter = (epoch - 1) * len(train_loader)
    for i, data in enumerate(train_loader):
        images, labels = data
        N = images.size(0)
        images = Variable(images).to(device)
        labels = Variable(labels).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        outputs = outputs[0]
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        prediction = outputs.max(1, keepdim=True)[1]
        train_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)
        train_loss.update(loss.item())
        curr_iter += 1
        if (i + 1) % 100 == 0:
            print('[epoch %d], [iter %d / %d], [train loss %.5f], [train acc %.5f]' % (
                epoch, i + 1, len(train_loader), train_loss.avg, train_acc.avg))
            total_loss_train.append(train_loss.avg)
            total_acc_train.append(train_acc.avg)
    return train_loss.avg, train_acc.avg

In [ ]:
def validate(val_loader, model, criterion, optimizer, epoch):
    model.eval()
    val_loss = AverageMeter()
    val_acc = AverageMeter()
    with torch.no_grad():
        for i, data in tqdm(enumerate(val_loader)):
            images, labels = data
            N = images.size(0)
            images = Variable(images).to(device)
            labels = Variable(labels).to(device)

            outputs = model(images)
            prediction = outputs.max(1, keepdim=True)[1]

            val_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)

            val_loss.update(criterion(outputs, labels).item())

    print('------------------------------------------------------------')
    print('[epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss.avg, val_acc.avg))
    print('------------------------------------------------------------')
    return val_loss.avg, val_acc.avg

In [ ]:
epoch_num = 10
best_val_acc = 0
total_loss_val, total_acc_val = [], []

for epoch in tqdm(range(1, epoch_num + 1)):
    loss_train, acc_train = train(train_loader, model, criterion, optimizer, epoch)
    loss_val, acc_val = validate(val_loader, model, criterion, optimizer, epoch)
    total_loss_val.append(loss_val)
    total_acc_val.append(acc_val)
    scheduler.step(loss_val) 
    if acc_val > best_val_acc:
        best_val_acc = acc_val
        print('*****************************************************')
        print('best record: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, loss_val, acc_val))
        print('*****************************************************')


In [ ]:
fig = plt.figure(num = 1)
fig2 = fig.add_subplot(1,1,1)
fig2.plot(total_acc_val, label = 'validation accuracy')
fig2.plot(total_loss_val, label = 'validation loss')

plt.legend()
plt.show()

In [ ]:
fig = plt.figure(num=1)
fig1 = fig.add_subplot(1,1,1)
fig1.plot(total_acc_train, label = 'training accuracy')
fig1.plot(total_loss_train, label = 'training loss')

plt.legend()
plt.show()

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Validation data evaluation

model.eval()
y_label = []
y_predict = []
with torch.no_grad():
    for i, data in enumerate(val_loader):
        images, labels = data
        N = images.size(0)
        images = Variable(images).to(device)
        outputs = model(images)
        prediction = outputs.max(1, keepdim=True)[1]
        y_label.extend(labels.cpu().numpy())
        y_predict.extend(np.squeeze(prediction.cpu().numpy().T))

# compute the confusion matrix
confusion_mtx = confusion_matrix(y_label, y_predict)
# plot the confusion matrix
plot_labels = ['akiec', 'bcc', 'bkl', 'df', 'nv', 'vasc','mel']
plot_confusion_matrix(confusion_mtx, plot_labels)

In [ ]:
# Test data evaluation

model.eval()
test_y_label = []
test_y_predict = []
with torch.no_grad():
    for i, data in enumerate(test_loader):
        images, labels = data
        N = images.size(0)
        images = Variable(images).to(device)
        outputs = model(images)
        prediction = outputs.max(1, keepdim=True)[1]
        test_y_label.extend(labels.cpu().numpy())
        test_y_predict.extend(np.squeeze(prediction.cpu().numpy().T))

# compute the confusion matrix
confusion_mtx_test = confusion_matrix(test_y_label, test_y_predict)
# plot the confusion matrix
plot_labels = ['akiec', 'bcc', 'bkl', 'df', 'nv', 'vasc','mel']
plot_confusion_matrix(confusion_mtx, plot_labels)

In [ ]:
# Generate a validation classification report
report = classification_report(y_label, y_predict, target_names=plot_labels)
print(report)

In [ ]:
# Generate a test classification report
report = classification_report(test_y_label, test_y_predict, target_names=plot_labels)
print(report)

In [ ]:
label_frac_error = 1 - np.diag(confusion_mtx) / np.sum(confusion_mtx, axis=1)
plt.bar(np.arange(7),label_frac_error)
plt.xlabel('True Label')
plt.ylabel('Fraction classified incorrectly')

In [ ]:
label_frac_error = 1 - np.diag(confusion_mtx_test) / np.sum(confusion_mtx_test, axis=1)
plt.bar(np.arange(7),label_frac_error)
plt.xlabel('True Label')
plt.ylabel('Fraction classified incorrectly')

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import torch
import numpy as np
from sklearn.preprocessing import label_binarize

def plot_roc_for_classes(model, val_loader, num_classes, device):
    """
    Fonction pour tracer la courbe ROC pour chaque classe.
    :param model: Modèle PyTorch.
    :param val_loader: DataLoader pour l'ensemble de validation.
    :param num_classes: Nombre de classes.
    :param device: Le périphérique (GPU ou CPU).
    """
    # Initialisation des labels et des probabilités prédites
    y_true = []
    y_scores = []

    model.eval()  # Mettre le modèle en mode évaluation

    with torch.no_grad():  # Pas besoin de calculer les gradients pendant l'évaluation
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Passer les images à travers le modèle
            outputs = model(images)
            
            # Calculer les probabilités via softmax
            probabilities = torch.softmax(outputs, dim=1)
            
            # Ajouter les véritables labels et les probabilités prédites
            y_true.extend(labels.cpu().numpy())
            y_scores.extend(probabilities.cpu().numpy())

    # Convertir les labels en format binaire (1 par classe)
    y_true_bin = label_binarize(y_true, classes=np.arange(num_classes))

    # Tracer la courbe ROC pour chaque classe
    plt.figure(figsize=(10, 8))
    for i in range(num_classes):
        # Calculer la courbe ROC
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], np.array([score[i] for score in y_scores]))
        roc_auc = auc(fpr, tpr)
        
        # Tracer la courbe ROC
        plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc:.2f})')
    
    # Tracer la ligne de non-séparation
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')

    # Ajouter les labels et légende
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) - Chaque Classe')
    plt.legend(loc='lower right')

    # Afficher le graphique
    plt.show()

# Exemple d'utilisation (après l'évaluation du modèle)
num_classes = 7  # Nombre de classes dans votre dataset
plot_roc_for_classes(model, val_loader, num_classes, device)


In [ ]:
torch.save(model.state_dict(), '/kaggle/working/model39.pth')
